## 借金の少ない会社を見つける

### Todo 

+ 会社リストで、一番底の跡に株価がどのように戻ったか、確認。
+ セクターごとに見る
+ ランキングを入れる


In [ ]:
import numpy as np

# Import Pipeline class and datasets
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data import USEquityPricing

from quantopian.pipeline.data.morningstar import Fundamentals

from quantopian.pipeline.data.factset import Fundamentals as factst_funda
from quantopian.pipeline.filters import Q500US, StaticSids, StaticAssets

# Import built-in trading universe
from quantopian.pipeline.filters import QTradableStocksUS


def make_pipeline():
    # Create a reference to our trading universe
    base_universe = QTradableStocksUS()

    # Get latest closing price
    close_price = USEquityPricing.close.latest
    
    #current_liabilities
    #The debts or obligations of the firm that are due within one year.
    current_liabilities = Fundamentals.current_liabilities.latest
    
    #total_assets
    #The aggregate amount of probable future economic benefits obtained or controlled by a particular enterprise as a result of past transactions or events.
    total_assets = Fundamentals.total_assets.latest
    market_cap = Fundamentals.market_cap.latest
    morningstar_industry_group_code = Fundamentals.morningstar_industry_group_code.latest
    morningstar_industry_code = Fundamentals.morningstar_industry_code.latest
    total_liabilities_net_minority_interest = Fundamentals.total_liabilities_net_minority_interest.latest
    operating_income = Fundamentals.operating_income.latest
    interest_expense = Fundamentals.interest_expense.latest
    cash_and_cash_equivalents = Fundamentals.cash_and_cash_equivalents.latest
    current_assets = Fundamentals.current_assets.latest
    current_liabilities = Fundamentals.current_liabilities.latest
    current_ratio = Fundamentals.current_ratio.latest
    
    # interest_expense

    myassets = StaticAssets(symbols(["PETS","SLP"]))#'AAPL', 'FB', 'MSFT', 'AMZN', 'GOOG', 'KO'
    
    # Return Pipeline containing close_price and
    # sentiment_score that has our trading universe as screen
    return Pipeline(
        columns={
            'close_price': close_price,
#             'current_liabilities':current_liabilities, 
            'total_assets':total_assets, 
            'cash_and_cash_equivalents':cash_and_cash_equivalents,
            'current_assets':current_assets,
            'liab/asset':total_liabilities_net_minority_interest/total_assets,
            'cash/asset':cash_and_cash_equivalents/total_assets,
            'market_cap':market_cap / 1000000000,
            'morningstar_industry_group_code':morningstar_industry_group_code,
            'morningstar_industry_code':morningstar_industry_code,
            'total_liabilities_net_minority_interest': total_liabilities_net_minority_interest, 
            'operating_income':operating_income, 
            'interest_expense':interest_expense,
            'current_liabilities':current_liabilities, 
            'current_ratio':current_ratio,
            
        },
        screen=base_universe #base_universe #myassets
    )

# Import run_pipeline method
from quantopian.research import run_pipeline

# Execute pipeline created by make_pipeline
# between start_date and end_date
pipeline_output = run_pipeline(
    make_pipeline(),
    start_date='2019-01-01',
    end_date='2019-03-30'
)

# Display last 10 rows
pipeline_output.tail(10)

In [ ]:
liab_asset = pipeline_output.loc["2020-04-01"][["close_price", "liab/asset", "market_cap", "morningstar_industry_group_code","morningstar_industry_code", "operating_income", "interest_expense"]].sort_values(by="liab/asset")

In [ ]:
liab_asset.dropna().groupby("morningstar_industry_group_code")[["liab/asset", "market_cap", "operating_income"]].mean().sort_values(by="liab/asset")

In [ ]:
by_group = list(liab_asset[(liab_asset["liab/asset"]<0.20) &
           (liab_asset["operating_income"]>0) &
           (liab_asset["morningstar_industry_group_code"]!=10420)& 
           (liab_asset["morningstar_industry_group_code"]!=10310)&                 
           (liab_asset["morningstar_industry_group_code"]!=10150)                
                          ].sort_values(by="liab/asset").dropna().groupby(by="morningstar_industry_group_code"))

In [ ]:
len(liab_asset[(liab_asset["liab/asset"]<0.20) &
           (liab_asset["operating_income"]>0) &
           (liab_asset["morningstar_industry_group_code"]!=10420)& 
           (liab_asset["morningstar_industry_group_code"]!=10310)&                 
           (liab_asset["morningstar_industry_group_code"]!=10150)].sort_values(by="liab/asset").dropna())

In [ ]:
[code for code, df in by_group]

In [ ]:
# 6 は外した。バイオだから
by_group[1][1]

## 資産の中に現金及び現金同等物が含まれている比率

+ [current_ratio](https://www.quantopian.com/docs/data-reference/morningstar_fundamentals#current-ratio)：Current Asst / Current Liabilities . 2以上（流動性の借り入れの二倍以上の流動性現金等をもっている）
+ operating_income	＞ Positive 
+ 'cash/asset' > 0.5 



In [ ]:
df_current_asset_lib = pipeline_output.loc["2019-04-01"]
df_current_asset_lib = df_current_asset_lib[(df_current_asset_lib["current_ratio"] >2) & 
                                            (df_current_asset_lib["operating_income"] >0) &
                                            (df_current_asset_lib['cash/asset'] > 0.5 )
                                           ].sort_values(by=["current_ratio",  'cash/asset'], ascending=False)
df_current_asset_lib